In [12]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
spark = SparkSession.builder.appName('tree_methods').getOrCreate()
spark

In [4]:
df = spark.read.format('libsvm').load('sample_libsvm_data.txt')
df.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [5]:
df.count()

100

In [6]:
df.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [7]:
df.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                100|
|   mean|               0.57|
| stddev|0.49756985195624287|
|    min|                0.0|
|    max|                1.0|
+-------+-------------------+



In [8]:
train_df, test_df = df.randomSplit([0.7,0.3])

In [9]:
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier(numTrees=100)
gbc = GBTClassifier()

In [10]:
dt_model = dtc.fit(train_df)
rf_model = rfc.fit(train_df)
gb_model = gbc.fit(train_df)

In [11]:
dt_preds = dt_model.transform(test_df)
rf_preds = rf_model.transform(test_df)
gb_preds = gb_model.transform(test_df)

In [13]:
dt_preds.show()

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[121,122,123...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[125,126,127...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[150,151,152...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[151,152,153...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[152,153,154...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(69

In [14]:
evaluator = MulticlassClassificationEvaluator(metricName='accuracy')
evaluator

MulticlassClassificationEvaluator_d6a8c878e002

In [15]:
dt_acc = evaluator.evaluate(dt_preds)
rf_acc = evaluator.evaluate(rf_preds)
gb_acc = evaluator.evaluate(gb_preds)

In [16]:
print("DT Accuracy:", dt_acc)
print("RF Accuracy:", rf_acc)
print("GB Accuracy:", gb_acc)

DT Accuracy: 0.90625
RF Accuracy: 0.96875
GB Accuracy: 0.90625
